## Importando ferramentas

In [4]:
import spacy
from spacy import displacy
import pandas as pd 
import string 
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import os
import gc

## Dataset

In [5]:
articles = pd.read_csv('/home/renato/Documentos/Datasets/articles.csv')

In [6]:
articles.head()

,title,text,date,category,subcategory,link
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/10/1...


## Extração de textos relevantes
Para análise será utilizado somente o títulos das notícias

In [7]:
titles = articles['title']
del articles
gc.collect()

20

In [8]:
titles.head()

0    Lula diz que está 'lascado', mas que ainda tem...
1    'Decidi ser escrava das mulheres que sofrem', ...
2    Três reportagens da Folha ganham Prêmio Petrob...
3    Filme 'Star Wars: Os Últimos Jedi' ganha trail...
4    CBSS inicia acordos com fintechs e quer 30% do...
Name: title, dtype: object

## Modelo estatístico com suporte para a  Lígua Portuguesa

`python3 -m spacy download pt_core_news_sm`


In [9]:
nlp = spacy.load('pt_core_news_sm')

### Tokenização


In [10]:
test_text = titles[125]
print(test_text)

Com gol aos 50 min do 2º tempo, Egito vence e se classifica para a Copa-2018


In [13]:
doc = nlp(test_text)

token_list = []
for token in doc:
    token_list.append(token.text)
print(token_list)

['Com', 'gol', 'aos', '50', 'min', 'do', '2º', 'tempo', ',', 'Egito', 'vence', 'e', 'se', 'classifica', 'para', 'a', 'Copa-2018']


### Remoção de Stop Word

In [14]:


pt_spacy_stopwords = spacy.lang.pt.stop_words.STOP_WORDS



In [19]:
doc = nlp(test_text)

# filtering stop words
filtered_title = []
for word in doc:
    if word.is_stop == False:
        filtered_title.append(word)
print(f'Original Title: {test_text}\n')
print(f'Filter Title: {filtered_title}')

Original Title: Com gol aos 50 min do 2º tempo, Egito vence e se classifica para a Copa-2018

Filter Title: [gol, 50, min, 2º, ,, Egito, vence, e, classifica, a, Copa-2018]


### Lemmatização - Normalização Lexical

In [20]:
# finding lemma for each word
for word in doc:
    print(word.text, word.lemma_)

Com Com
gol gol
aos aos
50 50
min min
do do
2º 2º
tempo tempo
, ,
Egito Egito
vence vencer
e e
se se
classifica classificar
para parir
a o
Copa-2018 Copa-2018


### Part of Speech (POS) Tagging

In [21]:
for word in doc:
    print(word.text, word.pos_)

Com ADP
gol NOUN
aos ADP
50 NUM
min NOUN
do DET
2º ADJ
tempo NOUN
, PUNCT
Egito PROPN
vence VERB
e CCONJ
se PRON
classifica VERB
para ADP
a DET
Copa-2018 PROPN


In [22]:
displacy.render(doc, style='dep')

### Representação Vetorial

In [24]:
print(doc.vector.shape)

(96,)


In [25]:
print(doc.vector)

[-0.57160145  0.51019347 -0.05318034 -1.4567363   1.7118582   0.07948475
 -0.84597844 -2.7788715   3.2007434  -0.4590931  -0.21069674  0.7978762
  1.2622988   2.1680558   0.33618984 -1.0133982   0.64089    -0.10490454
 -2.3037107   1.1573586   2.1608377   2.099376    1.4837372   1.6828076
 -1.3952034   2.460684    2.3769867  -1.7115828   3.6980793  -1.526282
 -0.6266858  -1.5741748  -1.7478446   0.85100454  2.201788   -1.5485375
  2.81914     1.7960856  -1.6263628  -2.1875834   0.63805985  0.93607324
 -1.1544945  -0.09995326  0.21919857 -1.3240733   0.48773944 -3.3218403
  1.331499   -1.1807832  -0.45794785 -1.4854857   0.66454715 -1.6454402
 -0.90822065  0.16942757  0.795574    1.3160138   0.37106586 -2.2276175
 -0.56400746  2.6720972  -0.7538465  -2.338119    0.05621624  0.96709746
  1.1517409  -3.094762    0.39936945  2.3941355   1.1707209  -0.71756446
 -2.7889793   0.05618754 -0.8061451  -0.62680995  1.8999001   1.0523338
 -3.3993077  -2.7733645   1.8841869  -1.9466614   2.6293588 

### Categorização de Notícias

In [26]:
articles = pd.read_csv('/home/renato/Documentos/Datasets/articles.csv')

In [27]:
df = articles[['title','category']]

In [28]:
del titles
gc.collect()
del articles
gc.collect()


0

In [30]:
df = df[(df['category'] == 'mercado') | (df['category'] == 'esporte')]

In [32]:
df.head()

,title,category
4,CBSS inicia acordos com fintechs e quer 30% do...,mercado
7,Tite diz querer seguir na seleção após o Mundi...,esporte
12,BNDES descarta irregularidades nos empréstimos...,mercado
13,Apple cita fornecedor francês antes de discuti...,mercado
14,"Volkswagen terá crescimento de 10% ao ano, diz...",mercado


In [34]:
punctuations = string.punctuation
def spacy_tokenizer(sentence):
    sentence = sentence.strip().lower()
    mytokens = nlp(sentence)
    mytokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens]
    mytokens = [word for word in  mytokens if word not in pt_spacy_stopwords and word not in punctuations]

    return mytokens 

In [38]:
spacy_tokenizer('EU sou o mestre dos magos!')

['o', 'mestre', 'mago']

## Machine learning - Classificação de Textos

In [40]:
df['category'].unique()

array(['mercado', 'esporte'], dtype=object)

In [41]:
le = preprocessing.LabelEncoder()
le.fit(df['category'])

LabelEncoder()

In [42]:
df['category_num'] = le.transform(df['category'])
df.head()

,title,category,category_num
4,CBSS inicia acordos com fintechs e quer 30% do...,mercado,1
7,Tite diz querer seguir na seleção após o Mundi...,esporte,0
12,BNDES descarta irregularidades nos empréstimos...,mercado,1
13,Apple cita fornecedor francês antes de discuti...,mercado,1
14,"Volkswagen terá crescimento de 10% ao ano, diz...",mercado,1


In [44]:
x = df['title']
y = df['category_num']

## TF-IDF

In [45]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=spacy_tokenizer)

In [46]:
x, _, y, _ = train_test_split(x, y, test_size=0.90)

In [47]:
x_tfidf =tfidf_vectorizer.fit_transform(x)
x_tfidf.shape

(4070, 5713)

In [48]:
x_train, x_test, y_train, y_test = train_test_split(x_tfidf, y, test_size=0.30)

In [49]:
classifier = RandomForestClassifier()

In [50]:
classifier.fit(x_train, y_train)

RandomForestClassifier()

In [51]:
predicted = classifier.predict(x_tfidf)

print(f'Model Accuracy: {metrics.accuracy_score(y,predicted)}')
print(f'Model F1-score: {metrics.f1_score(y, predicted)}')

Model Accuracy: 0.9714987714987715
Model F1-score: 0.9721955896452541
